In [1]:
import pandas as pd
import numpy as np
import os

import config as cfg
from utils.yahoo_downloader import YahooDownloader
import utils.helper_functions as hf

""" from stockstats import wrap

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import mutual_info_classif """

pd.options.mode.chained_assignment = None

In [2]:
folder_path = './db/'
file_name = 'ohlcv.pkl'
file_path = folder_path + file_name

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

if os.path.isfile(file_path):
    df = pd.read_pickle(file_path)

else: # 2 minutes
    df = YahooDownloader(cfg.start_date, cfg.end_date, cfg.tickers).fetch_data()        
    df.to_pickle(file_path)
    df.to_csv(folder_path + 'ohlcv.csv')

df.head()

,date,open,high,low,close,volume,tic,day
0,2000-01-03,48.000000,48.000000,47.250000,47.250000,16100.0,MT,0
1,2000-01-04,3.830000,3.900000,3.600000,3.680000,10424000.0,A2A.MI,1
2,2000-01-04,9.820000,9.820000,9.820000,9.820000,4250.0,ABCA.PA,1
3,2000-01-04,34.355453,34.348408,32.248322,32.769817,926084.0,AC.PA,1
4,2000-01-04,19.790001,20.590000,19.209999,20.580000,1449856.0,AF.PA,1


In [3]:
df['date'] = pd.to_datetime(df['date'])

In [4]:
print(df.dtypes)

date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume           float64
tic               object
day                int64
dtype: object


In [5]:
def add_supports_resistances(df: pd.DataFrame) -> pd.DataFrame:
    df['rolling_min'] = hf.get_rolling_min(df['low'], cfg.target_days)
    df['rolling_max'] = hf.get_rolling_max(df['high'], cfg.target_days)
    df['last_close'] = df['close'].shift(1)

    df['pivot'] = hf.get_pivot(df['rolling_max'], df['rolling_min'], df['last_close'])

    df = df.dropna()

    df['support1'] = hf.get_support1(df['pivot'], df['rolling_max'])
    df['support2'] = hf.get_support2(df['pivot'], df['rolling_max'], df['rolling_min'])
    df['resistance1'] = hf.get_resistance1(df['pivot'], df['rolling_min'])
    df['resistance2'] = hf.get_resistance2(df['pivot'], df['rolling_max'], df['rolling_min'])

    return df

In [6]:
def init_support_resistance(df: pd.DataFrame) -> list:
    support = df.iloc[0][cfg.support_column]
    resistance = df.iloc[0][cfg.resistance_column]
    return support, resistance

def update_support_resistance(row_df: pd.DataFrame) -> list:
    support = row_df.__getattribute__(cfg.support_column)
    resistance = row_df.__getattribute__(cfg.resistance_column)
    return support, resistance

def get_profit(df: pd.DataFrame) -> list:
    profit = 1
    buy_price = 0
    bought_days = 0
    is_bought = False

    support, resistance = init_support_resistance(df)

    for row in df.itertuples():
        last_close = row.last_close
        open = row.open

        if is_bought and (last_close > resistance or
                        last_close < support * cfg.support_sell_coef):
            is_bought = False
            sell_price = open
            profit *= (sell_price / buy_price) * cfg.fee_coef
            print(f'{row.tic} - {row.date}: sold at {sell_price}, operation profit {(sell_price / buy_price) * cfg.fee_coef}')
        elif not is_bought and (last_close < support or
                                last_close > resistance * cfg.resistance_buy_coef):
            is_bought = True
            buy_price = open
            profit *= cfg.fee_coef
            print(f'{row.tic} - {row.date}: bought at {buy_price}')

        if (last_close > resistance or last_close < support):
            support, resistance = update_support_resistance(row)

        if is_bought:
            bought_days += 1

    return profit - 1, bought_days

In [7]:
profits = []
available_tickers = df['tic'].unique()

for ticker in available_tickers:
    ticker_df = df.loc[df['tic'] == ticker]
    ticker_df = add_supports_resistances(ticker_df)

    days = len(ticker_df)

    if days > 0:
        profit, bought_days = get_profit(ticker_df)

        profits.append({
            'ticker': ticker,
            'profit': hf.pct(profit),
            'daily_profit': hf.pct(profit / bought_days) if bought_days > 0 else 0,
            'days': days,
            'bought_days': bought_days,
            'bought_days%': hf.pct(bought_days / days)
        })

profit_df = pd.DataFrame(profits)
print(profit_df.head())
print(f"Average profit: {profit_df['profit'].mean(skipna=True)}%")
print(f"Average daily profit %: {profit_df['daily_profit'].mean(skipna=True)}%")
print(f"Average bought days %: {profit_df['bought_days%'].mean(skipna=True)} days")

MT - 2000-01-28 00:00:00: bought at 42.75
MT - 2001-12-28 00:00:00: sold at 5.400000095367432, operation profit 0.12555789695427433
MT - 2002-02-14 00:00:00: bought at 4.5
MT - 2002-04-02 00:00:00: sold at 5.519999980926514, operation profit 1.2193066624535456
MT - 2004-01-13 00:00:00: bought at 22.200000762939453
MT - 2004-03-22 00:00:00: sold at 32.54999923706055, operation profit 1.4574188346718853
MT - 2004-10-14 00:00:00: bought at 81.75
MT - 2004-11-04 00:00:00: sold at 109.5, operation profit 1.3314128440366972
MT - 2005-04-15 00:00:00: bought at 82.19999694824219
MT - 2006-01-31 00:00:00: sold at 106.3499984741211, operation profit 1.2860328760089688
MT - 2006-05-16 00:00:00: bought at 110.8499984741211
MT - 2006-10-11 00:00:00: sold at 111.2699966430664, operation profit 0.9977661541333183
MT - 2007-08-13 00:00:00: bought at 177.92999267578125
MT - 2007-09-19 00:00:00: sold at 214.6199951171875, operation profit 1.1989674811891446
MT - 2007-11-20 00:00:00: bought at 211.259994

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(profit_df)

       ticker   profit  daily_profit  days  bought_days  bought_days%
0          MT   -98.22         -0.02  5772         3992         69.16
1      A2A.MI   -67.17         -0.02  5803         4311         74.29
2     ABCA.PA   -28.48         -0.01  5878         3545         60.31
3       AC.PA   177.25          0.04  5868         4695         80.01
4       AF.PA   -89.32         -0.02  5868         4035         68.76
5       AI.PA   637.53          0.15  5899         4323         73.28
6      AKW.PA   248.04          0.06  5898         4073         69.06
7     ALTA.PA   278.09          0.07  5864         3887         66.29
8       AM.PA    34.79          0.01  5899         3701         62.74
9      ATE.PA   124.02          0.03  5885         3697         62.82
10     ATO.PA   -49.44         -0.01  5884         4196         71.31
11     AVT.PA   -99.97         -0.02  5899         4720         80.01
12    BAMI.MI   -89.04         -0.02  5863         5064         86.37
13      BB.PA   -14.

In [9]:
cac_profit_df = profit_df[profit_df['ticker_year'].str.startswith('^FCHI')]
print(cac_profit_df)
print(f"Average profit: {cac_profit_df['profit'].mean(skipna=True)}%")
print(f"Average daily profit %: {cac_profit_df['daily_profit'].mean(skipna=True)}%")
print(f"Average bought days %: {cac_profit_df['bought_days%'].mean(skipna=True)} days")

KeyError: 'ticker_year'